# HTTP Header Security Check
This notebook checks the security headers of a given URL and allows the user to input cookies and verify a custom CORS setting.

In [1]:

import requests
from colorama import Fore, Style

def check_headers(url, cookies=None, custom_cors=None):
    security_headers = {
        "Strict-Transport-Security": "Missing or misconfigured. Should be set to 'max-age=31536000; includeSubDomains; preload'",
        "Content-Security-Policy": "Missing. Should be set to prevent XSS and other attacks.",
        "X-Frame-Options": "Missing. Should be 'DENY' or 'SAMEORIGIN' to prevent clickjacking.",
        "X-Content-Type-Options": "Missing. Should be 'nosniff' to prevent MIME type sniffing.",
        "Referrer-Policy": "Missing. Should be 'no-referrer' or 'strict-origin-when-cross-origin' to limit referrer information.",
        "Permissions-Policy": "Missing. Should restrict browser features like camera, microphone, and geolocation.",
        "Cache-Control": "Missing. Should be 'no-store' or 'max-age=0, no-cache, must-revalidate' to control caching.",
        "Set-Cookie": "Secure flag missing in cookies. Should have 'Secure; HttpOnly; SameSite=Strict' or 'Lax'.",
        "Access-Control-Allow-Origin": "Missing or misconfigured. Should be set to a specific trusted domain or 'none' to prevent unauthorized cross-origin access."
    }
    
    try:
        headers = {}
        if cookies:
            headers["Cookie"] = cookies
        if custom_cors:
            headers["Origin"] = custom_cors
        
        response = requests.get(url, headers=headers, timeout=10)
        response_headers = response.headers
        
        print(f"Checking security headers for: {url}\n")
        print("Received Headers:")
        for key, value in response_headers.items():
            print(f"{key}: {value}")
        print("\nSecurity Check Results:")
        
        for header, recommendation in security_headers.items():
            if header not in response_headers:
                print(Fore.RED + f"[!] {header} - {recommendation}" + Style.RESET_ALL)
            elif header == "Strict-Transport-Security" and "max-age" not in response_headers[header]:
                print(Fore.RED + f"[!] {header} - Incorrectly configured. {recommendation}" + Style.RESET_ALL)
            elif header == "Set-Cookie" and "Secure" not in response_headers.get("Set-Cookie", ""):
                print(Fore.RED + f"[!] {header} - Secure flag missing in cookies. {recommendation}" + Style.RESET_ALL)
            elif header == "Access-Control-Allow-Origin":
                if custom_cors and response_headers.get(header, "") != custom_cors:
                    print(Fore.RED + f"[!] {header} - Expected '{custom_cors}' but received '{response_headers.get(header, 'None')}'. {recommendation}" + Style.RESET_ALL)
                elif response_headers.get(header, "") == "*":
                    print(Fore.RED + f"[!] {header} - Wildcard '*' is insecure. {recommendation}" + Style.RESET_ALL)
                else:
                    print(Fore.GREEN + f"[+] {header} is properly configured." + Style.RESET_ALL)
            else:
                print(Fore.GREEN + f"[+] {header} is properly configured." + Style.RESET_ALL)
        
    except requests.RequestException as e:
        print(Fore.RED + f"Error fetching {url}: {e}" + Style.RESET_ALL)
    

In [ ]:

# User Inputs
target_url = input("Enter the URL to check: ").strip()
if not target_url.startswith("http://") and not target_url.startswith("https://"):
    # Add https:// prefix if missing
    target_url = "https://" + target_url
    
cookies = input("Enter cookies (if any, else press Enter to skip): ").strip()
cookies = cookies if cookies else None
custom_cors = input("Enter expected CORS value (if any, else press Enter to skip): ").strip()
custom_cors = custom_cors if custom_cors else None

# Run the security check
check_headers(target_url, cookies, custom_cors)


Checking security headers for: https://www.google.com

Received Headers:
Date: Wed, 05 Mar 2025 12:38:46 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=ISO-8859-1
Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce-1dItsjhokNw_GpjOLztGhg' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp
Accept-CH: Sec-CH-Prefers-Color-Scheme
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Content-Encoding: gzip
Server: gws
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Set-Cookie: AEC=AVcja2dLrjEJpFmnbKd8GDG1U6r-vlsxB4dnDLuX6vTbkGOgGlrpmpJBTA; expires=Mon, 01-Sep-2025 12:38:46 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax, NID=522=gNUXbT7SeCSgdYSL1Pd5Zi3yq37KaLBkCzAefDwlHBy996hrVfBgUEVp_TkOoWlNZd7XyMhTN0fl7FOXH9fajd5zyv5Sxm98qHt2F7ghAZzOu36a3fJjij-MqdlMGCR4QsLgZQub-YnAZeNtNyddq2V2v5JNwpVODjbPW5ZCBqPxyuBfv9mKlxV3